In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('Final_Train.csv')
test = pd.read_csv('Final_Test.csv')

In [ ]:
final = train.drop(['NEP Total' , 'Pediatric' , '15-24' , '25-34' , '35-44' , '45-54' , '55-64' , 'Geriatric'] , axis = 1)

In [ ]:
final = final.fillna(method ='bfill') 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaled = scaler.fit_transform(final.values)

In [ ]:
scaled_df = pd.DataFrame(scaled , columns = final.columns)

In [ ]:
scaled_df

,AGRA,AIZAWL,ALAPPUZHA,ALLAHABAD,ALWAR,AMRAVATI,AMRITSAR,BATHINDA,BHARUCH,BHOPAL,BULANDSHAHR,CACHAR,CHANDIGARH,CHENNAI,CHITTOOR,COIMBATORE,CUTTACK,DAKSHINAKANNADA,DEHRADUN,DHANBAD,DIBRUGARH,DURG,EASTKHASIHILLS,ERNAKULAM,FATEHGARHSAHIB,FIROZABAD,GANJAM,GHAZIABAD,GOLAGHAT,GUNTUR,GWALIOR,HASSAN,INDORE,JABALPUR,JAIPUR,JALANDHAR,JALGAON,JAMNAGAR,JHANSI,JODHPUR,...,LUDHIANA,MADURAI,MORADABAD,NAGPUR,NASHIK,NORTHGOA,PALAKKAD,PATIALA,PUNE,RAJKOT,RANCHI,RAYAGADA,SAGAR,SALEM,SANGLI,SIRMAUR,SOLAN,SOLAPUR,SONBHADRA,SONITPUR,SOUTHGOA,SURAT,THANE,THIRUVANANTHAPURAM,VADODARA,VALSAD,VARANASI,VISAKHAPATNAM,year,quarter,rainfall,relative_humidity,average_temperature,NSP Total/ NewMicConf Total,Relapse Total,NSN Total,so2,no2,rspm,SumOfHIV_infected
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000684,0.045074,0.452561,0.232995,0.288410,0.404624,0.179833,0.041652,0.020741,0.000000
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.039646,0.230408,0.845095,0.387310,0.336927,0.529865,0.169201,0.039022,0.017539,0.000000
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.096985,0.843237,0.782596,0.313198,0.312668,0.368015,0.161810,0.028246,0.012578,0.000000
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000210,0.488693,0.561346,0.331980,0.334232,0.332370,0.157474,0.049707,0.021652,0.000000
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.002314,0.189758,0.487903,0.348731,0.393531,0.349711,0.167749,0.036587,0.017993,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.9,1.000000,0.041512,0.844982,0.677657,0.376142,0.331536,0.237958,0.203658,0.028486,0.006297,0.228659
3368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000000,0.000342,0.471332,0.669981,0.448223,0.331536,0.274566,0.221259,0.034250,0.005970,0.429878
3369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error

In [ ]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df.drop('SumOfHIV_infected' , axis = 1), scaled_df['SumOfHIV_infected'], test_size=0.2, random_state=42)

In [ ]:
import numpy 

In [ ]:
X_train = X_train.values.reshape(2697 , 1 , 88)

In [ ]:
X_test = X_test.values.reshape(675 , 1 , 88)

In [ ]:
from tensorflow.keras import layers

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, 88)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(X_train , y_train , epochs = 50 , verbose = 2)

Epoch 1/50
85/85 - 0s - loss: 0.0166
Epoch 2/50
85/85 - 0s - loss: 0.0112
Epoch 3/50
85/85 - 0s - loss: 0.0079
Epoch 4/50
85/85 - 0s - loss: 0.0055
Epoch 5/50
85/85 - 0s - loss: 0.0041
Epoch 6/50
85/85 - 0s - loss: 0.0035
Epoch 7/50
85/85 - 0s - loss: 0.0032
Epoch 8/50
85/85 - 0s - loss: 0.0031
Epoch 9/50
85/85 - 0s - loss: 0.0029
Epoch 10/50
85/85 - 0s - loss: 0.0029
Epoch 11/50
85/85 - 0s - loss: 0.0028
Epoch 12/50
85/85 - 0s - loss: 0.0028
Epoch 13/50
85/85 - 0s - loss: 0.0027
Epoch 14/50
85/85 - 0s - loss: 0.0027
Epoch 15/50
85/85 - 0s - loss: 0.0027
Epoch 16/50
85/85 - 0s - loss: 0.0026
Epoch 17/50
85/85 - 0s - loss: 0.0026
Epoch 18/50
85/85 - 0s - loss: 0.0026
Epoch 19/50
85/85 - 0s - loss: 0.0026
Epoch 20/50
85/85 - 0s - loss: 0.0026
Epoch 21/50
85/85 - 0s - loss: 0.0025
Epoch 22/50
85/85 - 0s - loss: 0.0025
Epoch 23/50
85/85 - 0s - loss: 0.0025
Epoch 24/50
85/85 - 0s - loss: 0.0025
Epoch 25/50
85/85 - 0s - loss: 0.0025
Epoch 26/50
85/85 - 0s - loss: 0.0024
Epoch 27/50
85/85 - 0

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test , pred) #Vanilla LSTM

0.9056936331346117

In [ ]:
r2_score(y_test , y_pred) #Bidirectional RNN

0.8974135494608743

In [ ]:
r2_score(y_test , y_pred) #LSTM 1D

0.8974135494608743

In [ ]:
r2_score(y_test , y_pred) #LSTM 2D

0.8974135494608743

In [ ]:
from tensorflow.keras.layers import LSTM , Dropout , GRU , Bidirectional

In [ ]:
def create_model_bilstm(units):
    model = Sequential()
    model.add(Bidirectional(LSTM(units = units,                             
              return_sequences=True),
              input_shape=(1 , 88)))
    model.add(Bidirectional(LSTM(units = units)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

def create_model(units, m):
    model = Sequential()
    model.add(m (units = units, return_sequences = True,
                input_shape = (1 , 88)))
    model.add(Dropout(0.2))
    model.add(m (units = units))
    model.add(Dropout(0.2))
    model.add(Dense(units = 1))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
model_bilstm = create_model_bilstm(64)

In [ ]:
from tensorflow import keras
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 10)

In [ ]:
model_bilstm.fit(X_train, y_train, epochs = 100, validation_split = 0.2, batch_size = 32, shuffle = False, callbacks = [early_stop])

Epoch 1/100
68/68 [==============================] - 2s 26ms/step - loss: 0.0093 - val_loss: 0.0024
Epoch 2/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0022 - val_loss: 0.0018
Epoch 3/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 4/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 5/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 6/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 7/100
68/68 [==============================] - 0s 7ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 8/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 9/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0017 - val_loss: 0.0016
Epoch 10/100
68/68 [==============================] - 0s 6ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 11

In [ ]:
pred = model_bilstm.predict(X_test)

In [ ]:
r2_score(y_test , pred) #BiLSTM

0.9042291073163864

In [ ]:
model_gru = create_model(64, GRU)

In [ ]:
model_gru.fit(X_train, y_train, epochs = 100, validation_split = 0.2, batch_size = 32, shuffle = False, callbacks = [early_stop])

Epoch 1/100
68/68 [==============================] - 1s 13ms/step - loss: 0.0092 - val_loss: 0.0027
Epoch 2/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0038 - val_loss: 0.0019
Epoch 3/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0031 - val_loss: 0.0018
Epoch 4/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0029 - val_loss: 0.0018
Epoch 5/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0028 - val_loss: 0.0018
Epoch 6/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0028 - val_loss: 0.0018
Epoch 7/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0027 - val_loss: 0.0017
Epoch 8/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 9/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0024 - val_loss: 0.0017
Epoch 10/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0023 - val_loss: 0.0016
Epoch 11

In [ ]:
pred = model_gru.predict(X_test)

In [ ]:
r2_score(y_test , pred)

0.8964469769740756

In [ ]:
model_lstm = create_model(64, LSTM)

In [ ]:
model_lstm.fit(X_train, y_train, epochs = 100, validation_split = 0.2, batch_size = 32, shuffle = False, callbacks = [early_stop])

Epoch 1/100
68/68 [==============================] - 1s 14ms/step - loss: 0.0140 - val_loss: 0.0051
Epoch 2/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0036 - val_loss: 0.0020
Epoch 3/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0024 - val_loss: 0.0019
Epoch 4/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0025 - val_loss: 0.0018
Epoch 5/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0023 - val_loss: 0.0017
Epoch 6/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 7/100
68/68 [==============================] - 0s 4ms/step - loss: 0.0023 - val_loss: 0.0017
Epoch 8/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0022 - val_loss: 0.0017
Epoch 9/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 10/100
68/68 [==============================] - 0s 5ms/step - loss: 0.0022 - val_loss: 0.0016
Epoch 11

In [ ]:
pred = model_lstm.predict(X_test)

In [ ]:
r2_score(y_test , pred)

0.9003415647368461

In [ ]:
#PEDIATRIC

In [ ]:
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error

In [ ]:
r2_score(y_test , pred) #Pediatric Test

0.8272581422818612

In [ ]:
mean_absolute_error(y_test , pred)

0.03413500066861024

In [ ]:
mean_squared_error(y_test , pred)

0.002380809598005426

In [ ]:
#TRAIN

In [ ]:
Y_TRAIN_PRED = model.predict(X_train)

In [ ]:
r2_score(y_train , Y_TRAIN_PRED)

0.8663057849377733

In [ ]:
mean_absolute_error(y_train , Y_TRAIN_PRED)

0.03230865854419453

In [ ]:
mean_squared_error(y_train , Y_TRAIN_PRED)

0.002249948944071933

In [ ]:
#GERIATRIC

In [ ]:
pred = model.predict(X_test)

In [ ]:
r2_score(y_test , pred) #Geriatric Test

0.8700303759525237

In [ ]:
mean_absolute_error(y_test , pred)

0.03396449560656157

In [ ]:
mean_squared_error(y_test , pred)

0.0018805327749751438

In [ ]:
#Train

In [ ]:
Y_TRAIN_PRED = model.predict(X_train)

In [ ]:
r2_score(y_train , Y_TRAIN_PRED)

0.9011697369207206

In [ ]:
mean_absolute_error(y_train , Y_TRAIN_PRED)

0.029813304129812614

In [ ]:
mean_squared_error(y_train , Y_TRAIN_PRED)

0.0015773987181507863

In [ ]:
# HIV

In [ ]:
pred = model.predict(X_test)

In [ ]:
r2_score(y_test , pred) #HIV Test

0.8349750055331905

In [ ]:
mean_absolute_error(y_test , pred)

0.025265051817021725

In [ ]:
mean_squared_error(y_test , pred)

0.001867124927066518

In [ ]:
#TRAIN

In [ ]:
Y_TRAIN_PRED = model.predict(X_train)

In [ ]:
r2_score(y_train , Y_TRAIN_PRED)

0.8537044115912104

In [ ]:
mean_absolute_error(y_train , Y_TRAIN_PRED)

0.024875931270304057

In [ ]:
mean_squared_error(y_train , Y_TRAIN_PRED)

0.002063532698167419